<a href="https://colab.research.google.com/github/LilySu/BetterBusinessByReview/blob/master/NLP_Spacy_Named_Entity_Recognition_Parts_of_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [0]:
# !pip install scattertext
# !pip install flask_cors
# !pip install python-decouple
# !pip install -U spacy
# !python -m spacy download en_core_web_sm


In [0]:
from flask import Flask, render_template, request, jsonify
import json
import warnings
import pandas as pd
import spacy
import scattertext as st
from lxml import html
from requests import Session
from concurrent.futures import ThreadPoolExecutor as Executor
import requests
# from flask_cors import CORS
# from decouple import config
import re
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_colwidth', 1000)

In [0]:
nlp = spacy.load("en_core_web_sm")#if you run into problems here, 'Restart Runtime' and run all, it might fix things.

# Code


In [51]:
base_url = "https://www.yelp.com/biz/" 
api_url = "/review_feed?sort_by=date_desc&start="
bid = 'Rc1lxc5lSKJYd162JHNMfQ'

class Scraper():
    def __init__(self):
        self.data = pd.DataFrame()

    def get_data(self, n, bid=bid):
        with Session() as s:
            with s.get(base_url+bid+api_url+str(n*20)) as resp: #makes an http get request to given url and returns response as json
                r = json.loads(resp.content) #converts json response into a dictionary
                _html = html.fromstring(r['review_list']) #loads from dictionary

                dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                reviews = [el.text for el in _html.xpath("//div[@class='review-content']/p")]
                ratings = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")

                df = pd.DataFrame([dates, reviews, ratings]).T

                self.data = pd.concat([self.data,df])

    def scrape(self): #makes it faster
        # multithreaded looping
        with Executor(max_workers=40) as e:
            list(e.map(self.get_data, range(10)))

s = Scraper()
s.scrape()
df = s.data
# df = df.sample(100)

df.head(2)

,0,1,2
0,\n 12/26/2019\n,"Food portion to price ratio is what you would expect to be in upper manhattan. With the main entrees all at $15, I wasn't surprised the size would be small. Some plates however was not worth that price- the baked scrambled eggs was one of them. A few pieces of mozzarella and potatoes plus a small serving size of eggs was not worth that price for sure.",3.0 star rating
1,\n 12/21/2019\n,"My fiancé and I sat at the bar for a drink and a bite after visiting NYC for a show. Awesome place! The bartender was impressively attentive and efficient. We loved the draft beers, ricotta appetizer and amatriciana pizza. Many people ordered cocktails which Bea seems to be known for. The place was very busy at 5:15 on a Saturday so others must love this place as well!",5.0 star rating


In [52]:
def customtokensize(text):
    return re.findall("[\w']+", str(text))

df['tokenized_text'] = df[1].apply(customtokensize)
df.head(2)

,0,1,2,tokenized_text
0,\n 12/26/2019\n,"Food portion to price ratio is what you would expect to be in upper manhattan. With the main entrees all at $15, I wasn't surprised the size would be small. Some plates however was not worth that price- the baked scrambled eggs was one of them. A few pieces of mozzarella and potatoes plus a small serving size of eggs was not worth that price for sure.",3.0 star rating,"[Food, portion, to, price, ratio, is, what, you, would, expect, to, be, in, upper, manhattan, With, the, main, entrees, all, at, 15, I, wasn't, surprised, the, size, would, be, small, Some, plates, however, was, not, worth, that, price, the, baked, scrambled, eggs, was, one, of, them, A, few, pieces, of, mozzarella, and, potatoes, plus, a, small, serving, size, of, eggs, was, not, worth, that, price, for, sure]"
1,\n 12/21/2019\n,"My fiancé and I sat at the bar for a drink and a bite after visiting NYC for a show. Awesome place! The bartender was impressively attentive and efficient. We loved the draft beers, ricotta appetizer and amatriciana pizza. Many people ordered cocktails which Bea seems to be known for. The place was very busy at 5:15 on a Saturday so others must love this place as well!",5.0 star rating,"[My, fiancé, and, I, sat, at, the, bar, for, a, drink, and, a, bite, after, visiting, NYC, for, a, show, Awesome, place, The, bartender, was, impressively, attentive, and, efficient, We, loved, the, draft, beers, ricotta, appetizer, and, amatriciana, pizza, Many, people, ordered, cocktails, which, Bea, seems, to, be, known, for, The, place, was, very, busy, at, 5, 15, on, a, Saturday, so, others, must, love, this, place, as, well]"


In [53]:
stopwords = ['ve',"n't",'check-in','=','= =','u','want', 'u want', 'cuz','him',"i've",'on', 'her','told','ins', '1 check','I', 'i"m', 'i', ' ', 'it', "it's", 'it.','they', 'the', 'this','its', 'l','-','they','this',"don't",'the ', ' the', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','/','.',',']

def filter_stopwords(text):
  nonstopwords = []
  for i in text:
    if i not in stopwords:
      nonstopwords.append(i)
  return nonstopwords
df['tokenized_text'] = df['tokenized_text'].apply(filter_stopwords)
df['parts_of_speech_reference'] = df['tokenized_text'].apply(filter_stopwords)
df['parts_of_speech_reference'] = df['parts_of_speech_reference'].str.join(' ')
df.head(2)

,0,1,2,tokenized_text,parts_of_speech_reference
0,\n 12/26/2019\n,"Food portion to price ratio is what you would expect to be in upper manhattan. With the main entrees all at $15, I wasn't surprised the size would be small. Some plates however was not worth that price- the baked scrambled eggs was one of them. A few pieces of mozzarella and potatoes plus a small serving size of eggs was not worth that price for sure.",3.0 star rating,"[Food, portion, to, price, ratio, is, what, you, would, expect, to, be, in, upper, manhattan, With, main, entrees, all, at, 15, wasn't, surprised, size, would, be, small, Some, plates, however, was, not, worth, that, price, baked, scrambled, eggs, was, one, of, them, A, few, pieces, of, mozzarella, and, potatoes, plus, a, small, serving, size, of, eggs, was, not, worth, that, price, for, sure]",Food portion to price ratio is what you would expect to be in upper manhattan With main entrees all at 15 wasn't surprised size would be small Some plates however was not worth that price baked scrambled eggs was one of them A few pieces of mozzarella and potatoes plus a small serving size of eggs was not worth that price for sure
1,\n 12/21/2019\n,"My fiancé and I sat at the bar for a drink and a bite after visiting NYC for a show. Awesome place! The bartender was impressively attentive and efficient. We loved the draft beers, ricotta appetizer and amatriciana pizza. Many people ordered cocktails which Bea seems to be known for. The place was very busy at 5:15 on a Saturday so others must love this place as well!",5.0 star rating,"[My, fiancé, and, sat, at, bar, for, a, drink, and, a, bite, after, visiting, NYC, for, a, show, Awesome, place, The, bartender, was, impressively, attentive, and, efficient, We, loved, draft, beers, ricotta, appetizer, and, amatriciana, pizza, Many, people, ordered, cocktails, which, Bea, seems, to, be, known, for, The, place, was, very, busy, at, 15, a, Saturday, so, others, must, love, place, as, well]",My fiancé and sat at bar for a drink and a bite after visiting NYC for a show Awesome place The bartender was impressively attentive and efficient We loved draft beers ricotta appetizer and amatriciana pizza Many people ordered cocktails which Bea seems to be known for The place was very busy at 15 a Saturday so others must love place as well


In [54]:
def find_part_of_speech(x):
  """Use spacy's entity recognition to recogize if word is noun, verb, adjective, etc."""
  part_of_speech = []
  doc = nlp(str(x))
  for token in doc:
    part_of_speech.append(token.pos_)
  return part_of_speech

df['parts_of_speech'] = df['parts_of_speech_reference'].apply(find_part_of_speech)
df.head(2)

,0,1,2,tokenized_text,parts_of_speech_reference,parts_of_speech
0,\n 12/26/2019\n,"Food portion to price ratio is what you would expect to be in upper manhattan. With the main entrees all at $15, I wasn't surprised the size would be small. Some plates however was not worth that price- the baked scrambled eggs was one of them. A few pieces of mozzarella and potatoes plus a small serving size of eggs was not worth that price for sure.",3.0 star rating,"[Food, portion, to, price, ratio, is, what, you, would, expect, to, be, in, upper, manhattan, With, main, entrees, all, at, 15, wasn't, surprised, size, would, be, small, Some, plates, however, was, not, worth, that, price, baked, scrambled, eggs, was, one, of, them, A, few, pieces, of, mozzarella, and, potatoes, plus, a, small, serving, size, of, eggs, was, not, worth, that, price, for, sure]",Food portion to price ratio is what you would expect to be in upper manhattan With main entrees all at 15 wasn't surprised size would be small Some plates however was not worth that price baked scrambled eggs was one of them A few pieces of mozzarella and potatoes plus a small serving size of eggs was not worth that price for sure,"[NOUN, NOUN, PART, NOUN, NOUN, AUX, PRON, PRON, VERB, VERB, PART, AUX, ADP, ADJ, PROPN, ADP, ADJ, NOUN, DET, ADP, NUM, AUX, PART, ADJ, NOUN, VERB, AUX, ADJ, DET, NOUN, ADV, AUX, PART, ADJ, SCONJ, NOUN, VERB, VERB, NOUN, AUX, NUM, ADP, PRON, DET, ADJ, NOUN, ADP, NOUN, CCONJ, NOUN, CCONJ, DET, ADJ, VERB, NOUN, ADP, NOUN, AUX, PART, ADJ, DET, NOUN, ADP, ADJ]"
1,\n 12/21/2019\n,"My fiancé and I sat at the bar for a drink and a bite after visiting NYC for a show. Awesome place! The bartender was impressively attentive and efficient. We loved the draft beers, ricotta appetizer and amatriciana pizza. Many people ordered cocktails which Bea seems to be known for. The place was very busy at 5:15 on a Saturday so others must love this place as well!",5.0 star rating,"[My, fiancé, and, sat, at, bar, for, a, drink, and, a, bite, after, visiting, NYC, for, a, show, Awesome, place, The, bartender, was, impressively, attentive, and, efficient, We, loved, draft, beers, ricotta, appetizer, and, amatriciana, pizza, Many, people, ordered, cocktails, which, Bea, seems, to, be, known, for, The, place, was, very, busy, at, 15, a, Saturday, so, others, must, love, place, as, well]",My fiancé and sat at bar for a drink and a bite after visiting NYC for a show Awesome place The bartender was impressively attentive and efficient We loved draft beers ricotta appetizer and amatriciana pizza Many people ordered cocktails which Bea seems to be known for The place was very busy at 15 a Saturday so others must love place as well,"[DET, NOUN, CCONJ, VERB, ADP, NOUN, ADP, DET, NOUN, CCONJ, DET, NOUN, ADP, VERB, PROPN, ADP, DET, NOUN, PROPN, NOUN, DET, NOUN, AUX, ADV, ADJ, CCONJ, ADJ, PRON, VERB, NOUN, NOUN, PROPN, PROPN, CCONJ, NOUN, NOUN, ADJ, NOUN, VERB, NOUN, DET, PROPN, VERB, PART, AUX, VERB, ADP, DET, NOUN, AUX, ADV, ADJ, ADP, NUM, DET, PROPN, SCONJ, NOUN, VERB, VERB, NOUN, ADV, ADV]"


In [55]:
#Useless?
def extract_adjective_indexes(text):
  """Get the indexes of Adjectives and delete the occurrence
   of adjectives in order to persistently find new adjective 
   occurrences. In the future, add words occurring before and after"""
  adjective_indexes = []
  for i in text:
    if i == 'ADJ':
      adj_index = text.index('ADJ')
      adjective_indexes.append(adj_index)
      text.remove(i)
  return adjective_indexes

df['adjective_positions'] = df['parts_of_speech'].apply(extract_adjective_indexes)
df.head(2)

,0,1,2,tokenized_text,parts_of_speech_reference,parts_of_speech,adjective_positions
0,\n 12/26/2019\n,"Food portion to price ratio is what you would expect to be in upper manhattan. With the main entrees all at $15, I wasn't surprised the size would be small. Some plates however was not worth that price- the baked scrambled eggs was one of them. A few pieces of mozzarella and potatoes plus a small serving size of eggs was not worth that price for sure.",3.0 star rating,"[Food, portion, to, price, ratio, is, what, you, would, expect, to, be, in, upper, manhattan, With, main, entrees, all, at, 15, wasn't, surprised, size, would, be, small, Some, plates, however, was, not, worth, that, price, baked, scrambled, eggs, was, one, of, them, A, few, pieces, of, mozzarella, and, potatoes, plus, a, small, serving, size, of, eggs, was, not, worth, that, price, for, sure]",Food portion to price ratio is what you would expect to be in upper manhattan With main entrees all at 15 wasn't surprised size would be small Some plates however was not worth that price baked scrambled eggs was one of them A few pieces of mozzarella and potatoes plus a small serving size of eggs was not worth that price for sure,"[NOUN, NOUN, PART, NOUN, NOUN, AUX, PRON, PRON, VERB, VERB, PART, AUX, ADP, PROPN, ADP, NOUN, DET, ADP, NUM, AUX, PART, NOUN, VERB, AUX, DET, NOUN, ADV, AUX, PART, SCONJ, NOUN, VERB, VERB, NOUN, AUX, NUM, ADP, PRON, DET, NOUN, ADP, NOUN, CCONJ, NOUN, CCONJ, DET, VERB, NOUN, ADP, NOUN, AUX, PART, DET, NOUN, ADP]","[13, 15, 21, 24, 29, 39, 46, 52, 55]"
1,\n 12/21/2019\n,"My fiancé and I sat at the bar for a drink and a bite after visiting NYC for a show. Awesome place! The bartender was impressively attentive and efficient. We loved the draft beers, ricotta appetizer and amatriciana pizza. Many people ordered cocktails which Bea seems to be known for. The place was very busy at 5:15 on a Saturday so others must love this place as well!",5.0 star rating,"[My, fiancé, and, sat, at, bar, for, a, drink, and, a, bite, after, visiting, NYC, for, a, show, Awesome, place, The, bartender, was, impressively, attentive, and, efficient, We, loved, draft, beers, ricotta, appetizer, and, amatriciana, pizza, Many, people, ordered, cocktails, which, Bea, seems, to, be, known, for, The, place, was, very, busy, at, 15, a, Saturday, so, others, must, love, place, as, well]",My fiancé and sat at bar for a drink and a bite after visiting NYC for a show Awesome place The bartender was impressively attentive and efficient We loved draft beers ricotta appetizer and amatriciana pizza Many people ordered cocktails which Bea seems to be known for The place was very busy at 15 a Saturday so others must love place as well,"[DET, NOUN, CCONJ, VERB, ADP, NOUN, ADP, DET, NOUN, CCONJ, DET, NOUN, ADP, VERB, PROPN, ADP, DET, NOUN, PROPN, NOUN, DET, NOUN, AUX, ADV, CCONJ, PRON, VERB, NOUN, NOUN, PROPN, PROPN, CCONJ, NOUN, NOUN, NOUN, VERB, NOUN, DET, PROPN, VERB, PART, AUX, VERB, ADP, DET, NOUN, AUX, ADV, ADP, NUM, DET, PROPN, SCONJ, NOUN, VERB, VERB, NOUN, ADV, ADV]","[24, 25, 34, 48]"


In [56]:
def find_adj(x):
  """Get Just the Adjectives"""
  adj_list = []
  doc = nlp(str(x))
  for token in doc:
    if token.pos_ == 'ADJ':
      adj_list.append(token)
  return adj_list

df['adj_list'] = df['parts_of_speech_reference'].apply(find_adj)
df.head(2)

,0,1,2,tokenized_text,parts_of_speech_reference,parts_of_speech,adjective_positions,adj_list
0,\n 12/26/2019\n,"Food portion to price ratio is what you would expect to be in upper manhattan. With the main entrees all at $15, I wasn't surprised the size would be small. Some plates however was not worth that price- the baked scrambled eggs was one of them. A few pieces of mozzarella and potatoes plus a small serving size of eggs was not worth that price for sure.",3.0 star rating,"[Food, portion, to, price, ratio, is, what, you, would, expect, to, be, in, upper, manhattan, With, main, entrees, all, at, 15, wasn't, surprised, size, would, be, small, Some, plates, however, was, not, worth, that, price, baked, scrambled, eggs, was, one, of, them, A, few, pieces, of, mozzarella, and, potatoes, plus, a, small, serving, size, of, eggs, was, not, worth, that, price, for, sure]",Food portion to price ratio is what you would expect to be in upper manhattan With main entrees all at 15 wasn't surprised size would be small Some plates however was not worth that price baked scrambled eggs was one of them A few pieces of mozzarella and potatoes plus a small serving size of eggs was not worth that price for sure,"[NOUN, NOUN, PART, NOUN, NOUN, AUX, PRON, PRON, VERB, VERB, PART, AUX, ADP, PROPN, ADP, NOUN, DET, ADP, NUM, AUX, PART, NOUN, VERB, AUX, DET, NOUN, ADV, AUX, PART, SCONJ, NOUN, VERB, VERB, NOUN, AUX, NUM, ADP, PRON, DET, NOUN, ADP, NOUN, CCONJ, NOUN, CCONJ, DET, VERB, NOUN, ADP, NOUN, AUX, PART, DET, NOUN, ADP]","[13, 15, 21, 24, 29, 39, 46, 52, 55]","[upper, main, surprised, small, worth, few, small, worth, sure]"
1,\n 12/21/2019\n,"My fiancé and I sat at the bar for a drink and a bite after visiting NYC for a show. Awesome place! The bartender was impressively attentive and efficient. We loved the draft beers, ricotta appetizer and amatriciana pizza. Many people ordered cocktails which Bea seems to be known for. The place was very busy at 5:15 on a Saturday so others must love this place as well!",5.0 star rating,"[My, fiancé, and, sat, at, bar, for, a, drink, and, a, bite, after, visiting, NYC, for, a, show, Awesome, place, The, bartender, was, impressively, attentive, and, efficient, We, loved, draft, beers, ricotta, appetizer, and, amatriciana, pizza, Many, people, ordered, cocktails, which, Bea, seems, to, be, known, for, The, place, was, very, busy, at, 15, a, Saturday, so, others, must, love, place, as, well]",My fiancé and sat at bar for a drink and a bite after visiting NYC for a show Awesome place The bartender was impressively attentive and efficient We loved draft beers ricotta appetizer and amatriciana pizza Many people ordered cocktails which Bea seems to be known for The place was very busy at 15 a Saturday so others must love place as well,"[DET, NOUN, CCONJ, VERB, ADP, NOUN, ADP, DET, NOUN, CCONJ, DET, NOUN, ADP, VERB, PROPN, ADP, DET, NOUN, PROPN, NOUN, DET, NOUN, AUX, ADV, CCONJ, PRON, VERB, NOUN, NOUN, PROPN, PROPN, CCONJ, NOUN, NOUN, NOUN, VERB, NOUN, DET, PROPN, VERB, PART, AUX, VERB, ADP, DET, NOUN, AUX, ADV, ADP, NUM, DET, PROPN, SCONJ, NOUN, VERB, VERB, NOUN, ADV, ADV]","[24, 25, 34, 48]","[attentive, efficient, Many, busy]"


In [65]:
def find_phrases(x):
  """Create a list where adjectives come immediately before nouns for each review"""
  adj_list = []
  doc = nlp(str(x))
  try:
    for token in range(len(doc)):
      sub_list = []
      if (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='NOUN') or (doc[token].pos_ == 'VERB'and doc[token+1].pos_ =='NOUN'):
        sub_list.append(doc[token])
        sub_list.append(doc[token+1])
      elif (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'ADJ'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ =='VERB'and doc[token+2].pos_ =='NOUN')or (doc[token].pos_ == 'ADJ'and doc[token+1].pos_ == 'NOUN'and doc[token+2].pos_ =='NOUN'):
        sub_list.append(doc[token])
        sub_list.append(doc[token+1])        
        sub_list.append(doc[token+2])
      if (doc[token].lemma_ == 'wait'):
        sub_list.append(doc[token-2]) 
        sub_list.append(doc[token-1])
        sub_list.append(doc[token])
        sub_list.append(doc[token+1])
        sub_list.append(doc[token+2])
        sub_list.append(doc[token+3])
      if (doc[token].lemma_ == 'service'):
        sub_list.append(doc[token-2]) 
        sub_list.append(doc[token-1])
        sub_list.append(doc[token])
        sub_list.append(doc[token+1])
        sub_list.append(doc[token+2])
        sub_list.append(doc[token+3])
      if len(sub_list) != 0:
        adj_list.append(sub_list)
    return adj_list
  except IndexError as e:
    pass

df['adj_noun_phrases'] = df['parts_of_speech_reference'].apply(find_phrases)
df['adj_noun_phrases'].sample(200)

12                                                                                                                                                                                                                                                                                         []
16                                                                                                                                                                                                                                                                                         []
15                                                                                                                                                                             [[upscale, tavern, atmosphere], [tavern, atmosphere], [prompt, service], [and, prompt, service, at, bar, The]]
10                                                                                                                                            

In [58]:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Apple 0 5 ORG
U.K. 27 31 GPE
$1 billion 44 54 MONEY


In [61]:
def find_money(x):
  """Create a list where adjectives come immediately before nouns for each review"""
  money_list = []
  doc = nlp(str(x))
  for ent in doc.ents:
    if ent.label_ == 'MONEY':
      money_list.append(ent)
  return money_list

df['money_list'] = df['parts_of_speech_reference'].apply(find_money)
df['money_list'].sample(200)

11                                []
17                                []
4                                 []
8                                 []
9                                 []
12                                []
11                                []
5                                 []
1                                 []
3                                 []
16                                []
19                                []
12                                []
3                                 []
7                                 []
12                                []
7                                 []
12                                []
14                                []
5                                 []
16                                []
13                                []
18    [(26, dollars), (29, dollars)]
8                                 []
1                                 []
6                                 []
15                                []
1

In [63]:
def find_noun_chunks(x):
  """Create a list where adjectives come immediately before nouns for each review"""
  noun_list = []
  doc = nlp(str(x))
  for chunk in doc.noun_chunks:
      noun_list.append(chunk)
  return noun_list

df['noun_chunks'] = df['parts_of_speech_reference'].apply(find_noun_chunks)
df['noun_chunks'].sample(200)

2                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 [(brunch), (noon, place), (we), (their, indoor, outdoor, space), (The, vibe), (decor, pay, tribute), (American, classics), (a, relaxed, energy), (Sun), (wish), (sunglasses)]
16                                                                                                                                                                                                                                                                                                                      

In [64]:
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

Autonomous amod cars NOUN []
cars nsubj shift VERB [Autonomous]
shift ROOT shift VERB [cars, liability]
insurance compound liability NOUN []
liability dobj shift VERB [insurance, toward]
toward prep liability NOUN [manufacturers]
manufacturers pobj toward ADP []


In [0]:
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

###anything after in preparation to feed back into top 10 and bottom 10 endpoint

In [0]:
corpus = st.CorpusFromPandas(df, 
                          category_col=2, 
                          text_col=1,
                          nlp=nlp).build()

term_freq_df = corpus.get_term_freq_df()
term_freq_df['highratingscore'] = corpus.get_scaled_f_scores('5.0 star rating')

term_freq_df['poorratingscore'] = corpus.get_scaled_f_scores('1.0 star rating')
dh = term_freq_df.sort_values(by= 'highratingscore', ascending = False)
dh = dh[['highratingscore', 'poorratingscore']]
dh = dh.reset_index(drop=False)

In [0]:
dh = dh.rename(columns={'highratingscore':'score'})
dh = dh.drop(columns='poorratingscore')
positive_df = dh.head(10)
negative_df = dh.tail(10)
# word_df = pd.concat([positive_df, negative_df])
# word_df

In [0]:
results = {'positive': [{'term': pos_term, 'score': pos_score} for pos_term, pos_score in zip(positive_df['term'], positive_df['score'])], 'negative': [{'term': neg_term, 'score': neg_score} for neg_term, neg_score in zip(negative_df['term'], negative_df['score'])]}

In [0]:
results